In [30]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, \
                                    MaxPooling2D, BatchNormalization, Activation, \
                                    concatenate, Input, GlobalAveragePooling2D, \
                                    BatchNormalization, Flatten, Add, Input, InputLayer, \
                                    Activation, AveragePooling2D

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, \
                                        EarlyStopping

from tensorflow.keras import backend

from tensorflow.keras.regularizers import l1, l2

In [108]:
# height, width, number of channel
def build_model(input_shape=(36, 36, 3, 2), drop_rate=0.3, hidden_units=128, output_size=1):
    input_layer = Input(input_shape)
    cur_input_layer, next_input_layer = \
                                [tf.squeeze(layer, axis=-1) for layer in tf.split(t, num_or_size_splits=2, axis=-1)]
                                    
    print(input_layer)
    print(cur_input_layer, next_input_layer)
    # Appearance Model
    appearance_input_layer = cur_input_layer
    
    appr_layer1 = Conv2D(32, (3, 3), activation='tanh', padding='same')(appearance_input_layer)
    appr_layer2 = Conv2D(32, (3, 3), activation='tanh', padding='same')(appr_layer1)
    
    appr_layer3 = AveragePooling2D()(appr_layer2)
    appr_layer3 = Conv2D(32, (2, 2), activation='tanh', padding='same')(appr_layer3)
    
    appr_norm1 = Conv2D(32, (1, 1), activation='sigmoid', padding='same')(appr_layer3)
#     appr_norm1 = appr_norm1.shape[1] * appr_norm1.shape[2] * appr_norm1 / \
#                     (2 * backend.sum(appr_norm1, axis=2))
    
    
    appr_layer4 = Conv2D(32, (3, 3), activation='tanh', padding='same')(appr_layer3)
    appr_layer5 = Conv2D(32, (3, 3), activation='tanh', padding='same')(appr_layer4)
    
    appr_norm2 = Conv2D(32, (1, 1), activation='sigmoid', padding='same')(appr_layer5)
#     appr_norm2 = appr_norm2.shape[1] * appr_norm2.shape[2] * appr_norm2 / \
#                     (2 * appr_norm2.sum())
    
    # Motion Model
    # Layer 1
    motion_input_layer = (next_input_layer - cur_input_layer) / \
                            (next_input_layer + cur_input_layer)
    layer1 = Conv2D(32, (3, 3), activation='tanh', padding='same')(motion_input_layer)
    
    # Layer 2
    layer2 = Conv2D(32, (3, 3), activation='tanh', padding='same')(layer1)
    
    # normalization
#     normalization1 = 
    
    # Layer 3
    layer3 = AveragePooling2D()(layer2)
    layer3 = Conv2D(32, (2, 2), activation='tanh', padding='same')(layer3)
    
    # Layer 4
    layer4 = Conv2D(32, (3, 3), activation='tanh', padding='same')(layer3)
    
    # Layer 5
    layer5 = Conv2D(64, (3, 3), activation='tanh', padding='same')(layer4)
    print(layer5)
    
    # Layer 6
    layer6 = AveragePooling2D()(layer5)
    layer6 = Conv2D(64, (2, 2), activation='tanh', padding='same')(layer6)
    
    # Layer 7
    layer7 = Flatten()(layer6)
    
    # Layer 8
    layer8 = Dense(hidden_units, activation='relu')(layer7)    
    
    # Layer 9
    output_layer = Dense(output_size, activation='relu')(layer8)
    
    return Model(input_layer, output_layer)

In [109]:
model = build_model()
model.summary()

Tensor("input_21:0", shape=(?, 36, 36, 3, 2), dtype=float32)
Tensor("Squeeze_14:0", shape=(36, 36, 3), dtype=float32) Tensor("Squeeze_15:0", shape=(36, 36, 3), dtype=float32)


ValueError: Input 0 of layer conv2d_80 is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [36, 36, 3]

In [8]:
t = tf.constant([[[1.0], [1.0]], [[0.0], [1.0]]])

In [9]:
t.shape

TensorShape([Dimension(2), Dimension(2), Dimension(1)])

In [15]:
tf.Tensor(1, value_index=0, dtype=tf.int32)

TypeError: op needs to be an Operation: 1

In [80]:
t1 = tf.ones([36, 36, 3])
t2 = tf.ones([36, 36, 3])

In [29]:
tf.keras.backend.sum(t, axis=2)

<tf.Tensor 'Sum_4:0' shape=(36, 36) dtype=float32>

In [53]:
tf.Tensor([t1,t2], value_index=3, dtype=tf.int32)

TypeError: op needs to be an Operation: [<tf.Tensor 'ones_2:0' shape=(36, 36, 3) dtype=float32>, <tf.Tensor 'ones_3:0' shape=(36, 36, 3) dtype=float32>]

In [81]:
t1 = backend.expand_dims(t1, axis=-1)
t2 = backend.expand_dims(t2, axis=-1)

In [82]:
t1, t2

(<tf.Tensor 'ExpandDims_6:0' shape=(36, 36, 3, 1) dtype=float32>,
 <tf.Tensor 'ExpandDims_7:0' shape=(36, 36, 3, 1) dtype=float32>)

In [83]:
t = backend.concatenate((t1, t2), axis=-1)
print(t.shape)

(36, 36, 3, 2)


In [84]:
s = tf.split(t, num_or_size_splits=2, axis=-1)
print(s)

[<tf.Tensor 'split_5:0' shape=(36, 36, 3, 1) dtype=float32>, <tf.Tensor 'split_5:1' shape=(36, 36, 3, 1) dtype=float32>]


In [86]:
c1 , c2 = tf.split(t, num_or_size_splits=2, axis=-1)
print(c1.shape, c2.shape)

(36, 36, 3, 1) (36, 36, 3, 1)


In [107]:
tf.squeeze(c1, axis=-1)

<tf.Tensor 'Squeeze_13:0' shape=(36, 36, 3) dtype=float32>

In [41]:
input_layer = Input(input_shape)

In [40]:
input_shape=(2, 36, 36, 3)

In [42]:
input_layer.shape

TensorShape([Dimension(None), Dimension(2), Dimension(36), Dimension(36), Dimension(3)])